In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = 'X_2025-02.csv'
data_path = '/content/drive/MyDrive/05_Posgrados/Diplomado ciencia de datos/m2/proyecto/X_2025-02.csv'
df = pd.read_csv(data_path)

In [ ]:
df = df.sample(frac=.15)

In [ ]:
ID_COLS = ['FL_DATE', 'OP_UNIQUE_CARRIER','TAIL_NUM', 'ORIGIN', 'DEST',]
TARGET = 'DEP_DELAY_15'
features = [c for c in df.columns if c not in ID_COLS + [TARGET]]

In [ ]:
cols_with_nulls = (df.isnull().sum().sort_values()[df.isnull().sum().sort_values()>0]).index.tolist()

In [ ]:
features_model = [c for c in features if c not in cols_with_nulls]

In [ ]:
X = df[features_model]
y = df[TARGET]

In [ ]:
# Import models to train

# Modeling
from sklearn.model_selection import RandomizedSearchCV, train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [ ]:
# Partir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
base_model = LogisticRegression()
base_model.fit(X_train, y_train)

# Predict probabilities
y_scores = base_model.predict_proba(X_test)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_scores)
print(f"ROC AUC: {auc:.4f}")

ROC AUC: 0.6151


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Modelos a entrenar
random_state = 42
models = {
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=random_state),
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=random_state),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(random_state=random_state),
    'RandomForestClassifier': RandomForestClassifier(random_state=random_state)
}
param_grids = {
    'DecisionTreeClassifier': {
        'criterion': ['gini', 'entropy'],
        'max_depth': [5, 10],
        'min_samples_split': [2, 10],
        'min_samples_leaf': [1, 4],
        'class_weight': ['balanced', None]
    },
    'LogisticRegression': {
        'penalty': ['l2'],
        'C': [0.1, 1, 10],
        'solver': ['lbfgs'],
        'class_weight': ['balanced', None]
    },
    'KNeighborsClassifier': {
        'n_neighbors': [3, 7],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean']

    },
    'AdaBoostClassifier': {
        'n_estimators': [50],
        'learning_rate': [0.5, 1.0]

    },
    'RandomForestClassifier': {
        'n_estimators': [50, 100],
        'max_depth': [5, 10],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'class_weight': ['balanced']

    }
}

In [ ]:
# Resultados de la evaluación
results = []

for model_name, model in models.items():
    print(model_name)

    search = RandomizedSearchCV(model, param_grids[model_name], cv=5, scoring='roc_auc', n_jobs=-1,n_iter=100,)
    search.fit(X_train, y_train)

    # Obtener el mejor modelo y evaluar en el conjunto de prueba
    best_model = search.best_estimator_
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]

    # Evaluar el rendimiento
    report = classification_report(y_test, y_pred, output_dict=True)
    auc = roc_auc_score(y_test, y_pred_proba)

    results.append({
        'model': model_name,
        'best_params': search.best_params_,
        'classification_report': report,
        'roc_auc': auc
    })

DecisionTreeClassifier


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 32 is smaller than n_iter=100. Running 32 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


LogisticRegression


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=100. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


KNeighborsClassifier
AdaBoostClassifier


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 2 is smaller than n_iter=100. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined a

RandomForestClassifier


In [ ]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='roc_auc', ascending=False).reset_index(drop=True)
results_df

,model,best_params,classification_report,roc_auc
0,RandomForestClassifier,"{'n_estimators': 100, 'min_samples_split': 5, ...","{'0': {'precision': 0.8641083521444696, 'recal...",0.622443
1,DecisionTreeClassifier,"{'min_samples_split': 10, 'min_samples_leaf': ...","{'0': {'precision': 0.8127570789865872, 'recal...",0.618876
2,AdaBoostClassifier,"{'n_estimators': 50, 'learning_rate': 0.5}","{'0': {'precision': 0.8126638074815344, 'recal...",0.618627
3,LogisticRegression,"{'solver': 'lbfgs', 'penalty': 'l2', 'class_we...","{'0': {'precision': 0.8601522002867542, 'recal...",0.614568
4,KNeighborsClassifier,"{'weights': 'uniform', 'n_neighbors': 7, 'metr...","{'0': {'precision': 0.8163967231567757, 'recal...",0.565083


In [ ]:
# Train final model with all data

In [ ]:
results_df.iloc[0]['model']

'RandomForestClassifier'

In [ ]:

results_df.iloc[0]['classification_report']

{'0': {'precision': 0.8641083521444696,
  'recall': 0.5611668987759291,
  'f1-score': 0.680442607652313,
  'support': 13643.0},
 '1': {'precision': 0.2448284561049445,
  'recall': 0.6171701112877583,
  'f1-score': 0.35058249796803037,
  'support': 3145.0},
 'accuracy': 0.5716583273766976,
 'macro avg': {'precision': 0.554468404124707,
  'recall': 0.5891685050318437,
  'f1-score': 0.5155125528101717,
  'support': 16788.0},
 'weighted avg': {'precision': 0.7480948143171938,
  'recall': 0.5716583273766976,
  'f1-score': 0.6186478706403361,
  'support': 16788.0}}

In [ ]:
results_df.iloc[0]['best_params']

{'n_estimators': 100,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_depth': 10,
 'class_weight': 'balanced'}

In [ ]:
final_model = models[results_df.iloc[0]['model']]

In [ ]:
final_model.set_params(**results_df.iloc[0]['best_params'])

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [ ]:
final_model.fit(X, y)

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [ ]:
final_model

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [ ]:
pd.to_pickle(final_model, 'final_model.pkl')